# Module 2: Initial Price Push (Daily 6 AM Reset)

## Purpose
This module runs once daily at 8 AM Cairo time to:
1. **Load and prepare data** from Snowflake (MATERIALIZED_VIEWS.Pricing_data_extraction)
2. Reset prices for all SKUs based on ABC classification
3. Set initial cart rules based on normal_refill and stddev
4. Apply status-based adjustments (combined_status + yesterday_status)
5. Push cart rules and prices via API

## Data Flow
```
data_extraction.ipynb → Snowflake (Pricing_data_extraction) → Module 2 (this module)
                                                        ├── Data Preparation
                                                        ├── Price Logic
                                                        ├── Cart Rule Logic
                                                        └── Push to API
```

## Price Setting Logic
- **Zero demand SKUs**: Market minimum price + SKU discount
- **With market data**: A=25th percentile, B=50th, C=75th
- **Without market data**: A=50% margin range, B=75%, C=90%
- **No data SKUs**: Average margin of their category

## Status Adjustment Logic
- Both below On Track: -1 step from current price
- Both above On Track: +1 step from current price
- Combined lower, Yesterday higher: No action (oscillation prevention)
- Combined higher, Yesterday lower: No action (trend observation)
- On Track: No action
- Above On Track + Yesterday On Track: No action


In [1]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')

%run queries_module.ipynb
# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()
CURRENT_HOUR = CAIRO_NOW.hour

# Configuration constants
ABC_MARKET_PERCENTILES = {'A': 25, 'B': 50, 'C': 75}
ABC_MARGIN_PERCENTILES = {'A': 50, 'B': 75, 'C': 90}
# Cart rule constraints
MIN_CART_RULE = 10  # Minimum for Module 2 (fallback when no percentile data)
MIN_CART_RULE_OTHER = 5  # Minimum for other cases
MAX_CART_RULE = 500
LOW_STOCK_DOH_THRESHOLD = 2  # SKUs with DOH <= this are protected from price reduction
STATUS_BELOW_ON_TRACK = ['No Data', 'Critical', 'Struggling', 'Underperforming']
STATUS_ABOVE_ON_TRACK = ['Over Achiever', 'Star Performer']
STATUS_ON_TRACK = ['On Track']

# Input/Output configuration
# Data is now loaded from Snowflake instead of Excel
INPUT_TABLE = 'MATERIALIZED_VIEWS.Pricing_data_extraction'
OUTPUT_FILE = f'module_2_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

print(f"Module 2: Initial Price Push")
print(f"Run Time (Cairo): {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Input: {INPUT_TABLE} (today's data)")
print(f"Output: {OUTPUT_FILE}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron


/home/ec2-user/service_account_key.json


Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()
Module 2: Initial Price Push
Run Time (Cairo): 2026-02-18 08:09:59
Input: MATERIALIZED_VIEWS.Pricing_data_extraction (today's data)
Output: module_2_output_20260218_0809.xlsx


In [2]:
# =============================================================================
# LOAD DATA FROM SNOWFLAKE (Instead of Excel file)
# =============================================================================
print("Loading data from Snowflake...")

# Query to get today's data from Pricing_data_extraction
LOAD_QUERY = f"""
SELECT * FROM {INPUT_TABLE}
WHERE created_at = '{datetime.now(CAIRO_TZ).date()}'
"""

df_raw = query_snowflake(LOAD_QUERY)
print(f"Loaded {len(df_raw)} records from Snowflake")

# -------------------------------------------------------------------------
# DATA PREPARATION: Transform raw data to structured format
# This replicates the data structuring from pricing_action_engine.ipynb
# -------------------------------------------------------------------------
print("\nPreparing data (structuring columns, handling nulls)...")

# Create a clean DataFrame with all required columns and proper defaults
df = pd.DataFrame()

# Identifiers
df['warehouse_id'] = df_raw['warehouse_id']
df['product_id'] = df_raw['product_id']
df['sku'] = df_raw['sku']
df['cohort_id'] = df_raw['cohort_id'] if 'cohort_id' in df_raw.columns else None

# Product info
df['abc_class'] = df_raw['abc_class'].fillna('C')
df['brand'] = df_raw['brand'] if 'brand' in df_raw.columns else None
df['cat'] = df_raw['cat'] if 'cat' in df_raw.columns else None
df['sensitivity'] = df_raw['sensitivity'] if 'sensitivity' in df_raw.columns else None

# Current state - with null handling
df['current_price'] = pd.to_numeric(df_raw['current_price'], errors='coerce').fillna(0)
df['current_cart_rule'] = pd.to_numeric(df_raw['current_cart_rule'], errors='coerce').fillna(999)
df['normal_refill'] = pd.to_numeric(df_raw.get('normal_refill', 0), errors='coerce').fillna(0)
df['refill_stddev'] = pd.to_numeric(df_raw.get('refill_stddev', 0), errors='coerce').fillna(0)
df['wac_p'] = pd.to_numeric(df_raw['wac_p'], errors='coerce').fillna(0)
df['commercial_min_price'] = pd.to_numeric(df_raw.get('commercial_min_price', 0), errors='coerce').fillna(0)

# Performance status
df['combined_status'] = df_raw['combined_status'].fillna('No Data')
df['yesterday_status'] = df_raw['yesterday_status'].fillna('No Data')
df['oos_yesterday'] = df_raw['oos_yesterday'].fillna(0).astype(int)

# Stock and demand
df['stocks'] = pd.to_numeric(df_raw['stocks'], errors='coerce').fillna(0)
df['zero_demand'] = df_raw['zero_demand'].fillna(0).astype(int)
df['doh'] = pd.to_numeric(df_raw.get('doh', 999), errors='coerce').fillna(999)  # Days on Hand for low stock protection

# Margin data (for price tier calculations)
df['target_margin'] = pd.to_numeric(df_raw.get('target_margin', 0), errors='coerce').fillna(0)
#df['target_margin_std'] = pd.to_numeric(df_raw.get('target_margin_std', 0), errors='coerce').fillna(0)

# Market margins (for price tiers)
market_margin_cols = ['below_market', 'market_min', 'market_25', 'market_50', 
                      'market_75', 'market_max', 'above_market']
for col in market_margin_cols:
    if col in df_raw.columns:
        df[col] = pd.to_numeric(df_raw[col], errors='coerce')
    else:
        df[col] = np.nan

# Internal margin tiers
margin_tier_cols = ['margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3',
                    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']
for col in margin_tier_cols:
    if col in df_raw.columns:
        df[col] = pd.to_numeric(df_raw[col], errors='coerce')
    else:
        df[col] = np.nan

# All-time high margin (price ceiling for increases)
df['all_time_high_margin'] = pd.to_numeric(df_raw.get('all_time_high_margin', np.nan), errors='coerce')

# P80/P70 for cart rules fallback
df['p80_daily_240d'] = pd.to_numeric(df_raw.get('p80_daily_240d', 0), errors='coerce').fillna(0)
df['p70_daily_retailers_240d'] = pd.to_numeric(df_raw.get('p70_daily_retailers_240d', 1), errors='coerce').fillna(1)

print(f"✅ Data prepared: {len(df)} records")
print(f"\nABC Class Distribution:")
print(df['abc_class'].value_counts().to_string())
print(f"\nCombined Status Distribution:")
print(df['combined_status'].value_counts().to_string())

# =============================================================================
# LOAD PERCENTILE DATA FOR CART RULES
# =============================================================================
df_percentiles = get_percentile_data()


Loading data from Snowflake...


Loaded 28794 records from Snowflake

Preparing data (structuring columns, handling nulls)...
✅ Data prepared: 28794 records

ABC Class Distribution:
abc_class
C    23438
B     4656
A      700

Combined Status Distribution:
combined_status
No Data            7744
Struggling         5253
Underperforming    4698
On Track           3822
Critical           3347
Over Achiever      2200
Star Performer     1730
Fetching percentile data for cart rules...


  Loaded 17493 percentile records
   Percentiles available for 3379 unique products


In [3]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

# Minimum price change constant (ensure it's defined in this cell for function access)
MIN_PRICE_CHANGE_EGP = 0.25  # Minimum 0.25 EGP for any price increase or decrease

def is_below_on_track(status):
    """Check if status is below On Track."""
    return str(status).strip() in STATUS_BELOW_ON_TRACK

def is_above_on_track(status):
    """Check if status is above On Track."""
    return str(status).strip() in STATUS_ABOVE_ON_TRACK

def is_on_track(status):
    """Check if status is On Track."""
    return str(status).strip() in STATUS_ON_TRACK

def calculate_margin(price, wac):
    """Calculate margin from price and WAC."""
    if pd.isna(price) or pd.isna(wac) or price == 0:
        return None
    return (price - wac) / price

def get_market_tiers(row):
    """Get sorted list of market price tiers."""
    tiers = []
    for col in ['minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum']:
        val = row.get(col)
        if pd.notna(val) and val > 0:
            tiers.append(val)
    return sorted(set(tiers))

def get_margin_tiers(row):
    """Get sorted list of margin-based price tiers (converted to prices)."""
    tiers = []
    wac = row.get('wac_p', 0)
    if wac <= 0:
        return tiers
    
    for tier_col in ['margin_tier_below','margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
                     'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']:
        margin = row.get(tier_col)
        if pd.notna(margin) and 0 < margin < 1:
            price = wac / (1 - margin)
            tiers.append(round(price, 2))
    return sorted(set(tiers))

def find_next_price_above(current_price, row):
    """
    Find the first price tier ABOVE current_price by at least MIN_PRICE_CHANGE_EGP.
    Priority: Market tiers first, then margin tiers, then all_time_high_margin.
    Returns current_price if nothing found or already at ceiling.
    Skips tiers that are less than 0.25 EGP above current.
    """
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    wac = row.get('wac_p', 0)
    all_time_high_margin = row.get('all_time_high_margin', 0)
    
    # Calculate ceiling price from all_time_high_margin
    ceiling_price = None
    if pd.notna(all_time_high_margin) and all_time_high_margin > 0 and wac > 0:
        ceiling_price = wac / (1 - all_time_high_margin)
    
    # Check if already at or above ceiling
    if ceiling_price and current_price >= ceiling_price:
        return current_price  # Already at ceiling, no increase
    
    # Try market tiers first - skip tiers less than MIN_PRICE_CHANGE_EGP above current
    market_tiers = get_market_tiers(row)
    for tier in market_tiers:
        if tier > current_price + MIN_PRICE_CHANGE_EGP:  # Must be at least 0.25 EGP above
            # Ensure we don't exceed ceiling
            if ceiling_price and tier > ceiling_price:
                # Use ceiling if it's at least MIN_PRICE_CHANGE_EGP above current
                if ceiling_price > current_price + MIN_PRICE_CHANGE_EGP:
                    return round(ceiling_price, 2)
                return current_price
            return round(tier, 2)
    
    # Try margin tiers - skip tiers less than MIN_PRICE_CHANGE_EGP above current
    margin_tiers = get_margin_tiers(row)
    for tier in margin_tiers:
        if tier > current_price + MIN_PRICE_CHANGE_EGP:  # Must be at least 0.25 EGP above
            # Ensure we don't exceed ceiling
            if ceiling_price and tier > ceiling_price:
                if ceiling_price > current_price + MIN_PRICE_CHANGE_EGP:
                    return round(ceiling_price, 2)
                return current_price
            return round(tier, 2)
    
    # No tier found above - use all_time_high_margin as ceiling (fallback)
    if ceiling_price and ceiling_price > current_price + MIN_PRICE_CHANGE_EGP:
        return round(ceiling_price, 2)
    
    # Nothing found above with sufficient difference, keep current
    return current_price

def find_next_price_below(current_price, row):
    """
    Find the first price tier BELOW current_price by at least MIN_PRICE_CHANGE_EGP.
    Priority: Market tiers first, then margin tiers.
    Returns current_price if nothing found.
    Skips tiers that are less than 0.25 EGP below current.
    """
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    # Try market tiers first (reverse order to find highest below with sufficient diff)
    market_tiers = get_market_tiers(row)
    for tier in reversed(market_tiers):
        if tier < current_price - MIN_PRICE_CHANGE_EGP:  # Must be at least 0.25 EGP below
            return round(tier, 2)
    
    # Try margin tiers (reverse order) - skip tiers less than MIN_PRICE_CHANGE_EGP below
    if len(market_tiers) ==0:
        margin_tiers = get_margin_tiers(row)
        for tier in reversed(margin_tiers):
            if tier < current_price - MIN_PRICE_CHANGE_EGP:  # Must be at least 0.25 EGP below
                return round(tier, 2)
    
    # Nothing found below with sufficient difference, keep current
    return current_price

print("Helper functions loaded.")


Helper functions loaded.


In [4]:
# =============================================================================
# STATUS ADJUSTMENT & PRICE FUNCTIONS
# =============================================================================

def get_price_action(combined_status, yesterday_status):
    """
    Determine price action based on status combination.
    
    Returns:
        str: 'increase', 'decrease', or 'hold'
        str: Reason for action
    """
    combined_below = is_below_on_track(combined_status)
    combined_above = is_above_on_track(combined_status)
    combined_on = is_on_track(combined_status)
    yesterday_below = is_below_on_track(yesterday_status)
    yesterday_above = is_above_on_track(yesterday_status)
    yesterday_on = is_on_track(yesterday_status)
    
    # On Track = no action
    if combined_on and yesterday_on:
        return 'hold', "On Track - no price change"
    if combined_on and yesterday_above:
        return 'increase', f"yesterday above - combined on  ({combined_status}, {yesterday_status}) - increase"
    
    # Both ABOVE On Track: INCREASE price (only if both are above, not on track)
    if combined_above and yesterday_above:
        return 'increase', f"Both above ({combined_status}, {yesterday_status}) - increase"
    
    # Combined above, Yesterday on track: HOLD (changed from increase)
    if combined_above and yesterday_on:
        return 'hold', f"Above + On Track ({combined_status}, {yesterday_status}) - hold"
    
    # Both below On Track: DECREASE price (go to first tier below current)
    if combined_below and (yesterday_below or yesterday_on):
        return 'decrease', f"Both below/on ({combined_status}, {yesterday_status}) - decrease"
    
    # Combined below, Yesterday above: No action (oscillation prevention)
    if combined_below and yesterday_above:
        return 'hold', f"Oscillation prevention ({combined_status} vs {yesterday_status}) - hold"
    
    # Combined above, Yesterday below: HOLD (observe trend before reacting)
    if combined_above and yesterday_below:
        return 'hold', f"Trend observation ({combined_status} vs {yesterday_status}) - hold"
    
    return 'hold', "Default - no price change"

def apply_price_action(current_price, action, row):
    """
    Apply price action: find next tier above/below current price.
    
    Args:
        current_price: Current SKU price
        action: 'increase', 'decrease', or 'hold'
        row: DataFrame row with tier data
    
    Returns:
        float: New price
        str: Source of new price (market/margin/unchanged)
    """
    if action == 'hold' or pd.isna(current_price):
        return current_price, 'unchanged'
    
    if action == 'increase':
        # Check if already at or above all_time_high ceiling
        wac = row.get('wac_p', 0)
        all_time_high_margin = row.get('all_time_high_margin', 0)
        ceiling_price = None
        if pd.notna(all_time_high_margin) and all_time_high_margin > 0 and wac > 0:
            ceiling_price = wac / (1 - all_time_high_margin)
        
        # If current price is already at or above ceiling, HOLD
        if ceiling_price and current_price >= ceiling_price:
            return current_price, 'unchanged (at all_time_high ceiling)'
        
        new_price = find_next_price_above(current_price, row)
        if new_price > current_price:
            # Determine source
            market_tiers = get_market_tiers(row)
            if new_price in market_tiers:
                source = 'market'
            elif ceiling_price and abs(new_price - ceiling_price) < 0.0025:
                source = 'all_time_high_margin'
            else:
                source = 'margin'
            return new_price, source
        return current_price, 'unchanged (no tier above)'
    
    if action == 'decrease':
        new_price = find_next_price_below(current_price, row)
        if new_price < current_price:
            # Determine source
            market_tiers = get_market_tiers(row)
            source = 'market' if new_price in market_tiers else 'margin'
            
            # Apply commercial minimum floor
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            return new_price, source
        return current_price, 'unchanged (no tier below)'
    
    return current_price, 'unchanged'

def get_initial_cart_rule(row, percentile_data, is_oos=False, is_zero_demand=False):
    """
    Calculate initial cart rule using percentile-based approach.
    
    Primary: Use percentiles from historical order data
    - Normal case: 95th percentile
    - OOS: 95th percentile
    - Zero Demand: 95th percentile
    - Low Stock (DOH <= 2): 50th percentile
    
    Fallback: If percentile data unavailable, use 10 (minimum for Module 2)
    """
    cohort_id = row.get('cohort_id')
    product_id = row.get('product_id')
    
    # Try to get percentile data
    if len(percentile_data) > 0:
        percentile_row = percentile_data[
            (percentile_data['cohort_id'] == cohort_id) & 
            (percentile_data['product_id'] == product_id)
        ]
        
        if len(percentile_row) > 0:
            # Special case: OOS - Use 95th percentile
            if is_oos:
                perc_95 = percentile_row.iloc[0]['perc_95']
                if pd.notna(perc_95) and perc_95 > 0:
                    return max(MIN_CART_RULE, min(MAX_CART_RULE, int(round(perc_95))))
            
            # Special case: Zero Demand - Use 95th percentile
            if is_zero_demand:
                perc_95 = percentile_row.iloc[0]['perc_95']
                if pd.notna(perc_95) and perc_95 > 0:
                    return max(MIN_CART_RULE, min(MAX_CART_RULE, int(round(perc_95))))
            
            # Special case: Low Stock (DOH <= 2) - Use 50th percentile
            doh = row.get('doh', 999)
            if doh <= LOW_STOCK_DOH_THRESHOLD:
                perc_50 = percentile_row.iloc[0]['perc_50']
                if pd.notna(perc_50) and perc_50 > 0:
                    return max(MIN_CART_RULE, min(MAX_CART_RULE, int(round(perc_50))))
            
            # Normal case: Use 95th percentile
            perc_95 = percentile_row.iloc[0]['perc_95']
            if pd.notna(perc_95) and perc_95 > 0:
                return max(MIN_CART_RULE, min(MAX_CART_RULE, int(round(perc_95))))
    
    # Fallback: no percentile data available
    return MIN_CART_RULE

print("Status adjustment and price functions loaded.")


Status adjustment and price functions loaded.


In [5]:
# =============================================================================
# MAIN ENGINE: GENERATE INITIAL PRICE PUSH
# =============================================================================

def get_max_price(row):
    """Get maximum price: market_max first, then highest margin tier."""
    # Try market max first
    market_max = row.get('maximum')
    if pd.notna(market_max) and market_max > 0:
        return market_max, 'market_max'
    
    # Fallback: highest margin tier
    margin_tiers = get_margin_tiers(row)
    if margin_tiers:
        return margin_tiers[-1], 'margin_max'  # Last = highest
    
    # Fallback: current price
    return row.get('current_price', 0), 'unchanged'

def find_price_n_steps_below(current_price, n_steps, row):
    """Find price N steps below current (iteratively find next tier below)."""
    price = current_price
    for _ in range(n_steps):
        next_price = find_next_price_below(price, row)
        if next_price >= price:  # No tier below found
            break
        price = next_price
    return price

def generate_initial_price_push(row, percentile_data):
    """
    Generate initial price push action for a single SKU.
    
    Logic:
    - Stocks = 0: Set to market_max or margin_max (highest price)
    - Zero demand + yesterday below on track: Go 2 steps below current
    - Zero demand + yesterday above on track: Keep current price
    - Otherwise: Adjust price relative to CURRENT price based on status
    """
    result = {
        'product_id': row.get('product_id'),
        'warehouse_id': row.get('warehouse_id'),
        'cohort_id': row.get('cohort_id'),
        'sku': row.get('sku'),
        'brand': row.get('brand'),
        'cat': row.get('cat'),
        'abc_class': row.get('abc_class', 'C'),
        'current_price': row.get('current_price'),
        'current_cart_rule': row.get('current_cart_rule'),
        'wac_p': row.get('wac_p'),
        'stocks': row.get('stocks', 0),
        'combined_status': row.get('combined_status'),
        'yesterday_status': row.get('yesterday_status'),
        'zero_demand': row.get('zero_demand', 0),
        'sensitivity': row.get('sensitivity', row.get('product_sensitivity')),
        'new_price': None,
        'new_cart_rule': None,
        'new_margin': None,
        'current_margin': None,
        'price_source': None,
        'price_action': None,
        'price_reason': None,
    }
    
    wac = row.get('wac_p', 0)
    current_price = row.get('current_price', 0)
    result['current_margin'] = calculate_margin(current_price, wac)
    yesterday_status = row.get('yesterday_status', 'No Data')
    
    # CASE 1: Out of Stock (stocks = 0) - Set to MAX price, capped at target_margin + 3*std
    if row.get('stocks', 0) == 0:
        max_price, price_source = get_max_price(row)
        
        # Calculate cap price from target_margin + 3*std
        target_margin = row.get('target_margin', 0)
        margin_std = row.get('std', 0)  # margin std from data extraction
        cap_margin = target_margin + (5 * margin_std) if pd.notna(target_margin) and pd.notna(margin_std) else None
        cap_price = wac / (1 - cap_margin) if cap_margin and cap_margin < 1 and wac > 0 else None
        
        # Apply cap logic
        if cap_price and max_price > cap_price:
            # Max price exceeds cap - use max(current_price, cap_price)
            final_price = max(current_price, cap_price) if pd.notna(current_price) else cap_price
            result['new_price'] = round(final_price, 2)
            result['price_source'] = 'capped_at_target+3std'
            result['price_action'] = 'oos_capped'
            result['price_reason'] = f'OOS - max ({max_price:.2f}) > cap ({cap_price:.2f}), using max(current, cap)'
        else:
            # Max price is within cap or no cap - use max price
            result['new_price'] = max_price
            result['price_source'] = price_source
            result['price_action'] = 'oos_max'
            result['price_reason'] = 'OOS - set to max price (within cap)'
        
        result['new_cart_rule'] = get_initial_cart_rule(row, percentile_data, is_oos=True)  # OOS cart: 95th percentile
        result['new_margin'] = calculate_margin(result['new_price'], wac)
        return result
    
    # CASE 2: Zero Demand SKUs (has stock but no recent sales)
    if row.get('zero_demand', 0) == 1:
        yesterday_below = is_below_on_track(yesterday_status)
        yesterday_above = is_above_on_track(yesterday_status)
        
        if yesterday_below:
            # Yesterday below on track: Go 2 steps below current price
            new_price = find_price_n_steps_below(current_price, 2, row)
            
            # Apply commercial minimum floor
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            result['new_price'] = new_price
            result['price_source'] = 'market' if new_price in get_market_tiers(row) else 'margin'
            result['price_action'] = 'zero_demand_decrease'
            result['price_reason'] = f'Zero demand + yesterday below ({yesterday_status}) - 2 steps below'
        elif yesterday_above:
            # Yesterday above on track: Keep current price
            result['new_price'] = current_price
            result['price_source'] = 'unchanged'
            result['price_action'] = 'zero_demand_hold'
            result['price_reason'] = f'Zero demand + yesterday above ({yesterday_status}) - keep current'
        else:
            # Yesterday on track or no data: Go 1 step below
            new_price = find_next_price_below(current_price, row)
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            result['new_price'] = new_price
            result['price_source'] = 'market' if new_price in get_market_tiers(row) else 'margin'
            result['price_action'] = 'zero_demand_decrease'
            result['price_reason'] = f'Zero demand + yesterday on track ({yesterday_status}) - 1 step below'
        
        result['new_cart_rule'] = get_initial_cart_rule(row, percentile_data, is_zero_demand=True)  # Zero demand: 95th percentile
        result['new_margin'] = calculate_margin(result['new_price'], wac)
        return result
    
    # CASE 2.5: LOW STOCK PROTECTION (DOH <= 2 with demand)
    # Protect inventory until next receiving - no price reduction, cap cart at normal_refill
    # But still allow price INCREASE if status indicates strong performance
    doh = row.get('doh', 999)
    if doh <= LOW_STOCK_DOH_THRESHOLD and row.get('zero_demand', 0) == 0 and row.get('stocks', 0) > 0:
        combined_status = row.get('combined_status', 'No Data')
        normal_refill = row.get('normal_refill', 5) or 5
        
        # Check if we should increase price (status above on track)
        if is_above_on_track(combined_status):
            new_price = find_next_price_above(current_price, row)
            result['new_price'] = new_price
            result['price_source'] = 'market' if new_price in get_market_tiers(row) else 'margin'
            result['price_action'] = 'low_stock_increase'
            result['price_reason'] = f'Low stock (DOH={doh:.1f}) + above on track ({combined_status}) - increase allowed'
        else:
            # Hold price - no reduction for low stock
            result['new_price'] = current_price
            result['price_source'] = 'unchanged'
            result['price_action'] = 'low_stock_hold'
            result['price_reason'] = f'Low stock (DOH={doh:.1f}) - hold price (no reduction allowed)'
        
        # Cap cart at 50th percentile for low stock
        result['new_cart_rule'] = get_initial_cart_rule(row, percentile_data)  # Will use 50th percentile for low stock
        result['new_margin'] = calculate_margin(result['new_price'], wac)
        return result
    
    # CASE 3: Normal SKUs - Determine price action based on status
    combined_status = row.get('combined_status', 'No Data')
    
    # Handle 'No Data' with stocks as Critical (below on track)
    if combined_status == 'No Data' and row.get('stocks', 0) > 0:
        combined_status = 'Critical'
    
    # Get price action (increase/decrease/hold)
    action, reason = get_price_action(combined_status, yesterday_status)
    
    # Apply price action - find next tier above/below current price
    new_price, price_source = apply_price_action(current_price, action, row)
    
    result['new_price'] = new_price
    result['price_source'] = price_source
    result['price_action'] = action
    result['price_reason'] = reason
    result['new_cart_rule'] = get_initial_cart_rule(row, percentile_data)
    result['new_margin'] = calculate_margin(new_price, wac)
    
    return result

print("Main engine function loaded.")


Main engine function loaded.


In [6]:
# =============================================================================
# EXECUTE MODULE 2
# =============================================================================
print(f"Processing {len(df)} SKUs...")
print("="*60)

results = []
for idx, row in df.iterrows():
    result = generate_initial_price_push(row, df_percentiles)
    results.append(result)
    
    if (idx + 1) % 10000 == 0:
        print(f"Processed {idx + 1}/{len(df)} SKUs...")

df_results = pd.DataFrame(results)
print(f"\n✅ Processed {len(df_results)} SKUs")


Processing 28794 SKUs...


Processed 10000/28794 SKUs...


Processed 20000/28794 SKUs...



✅ Processed 28794 SKUs


In [7]:
# =============================================================================
# SUMMARY
# =============================================================================
print("="*60)
print("MODULE 2 SUMMARY")
print("="*60)

print(f"\nTotal SKUs processed: {len(df_results)}")
print(f"\nBy ABC Class:")
print(df_results['abc_class'].value_counts().to_string())
print(f"\nBy Price Source:")
print(df_results['price_source'].value_counts().to_string())

# Price change analysis
df_results['price_change'] = df_results['new_price'] - df_results['current_price']
df_results['price_change_pct'] = (df_results['price_change'] / df_results['current_price'] * 100).round(2)

print(f"\nPrice Change Distribution:")
print(f"  Increases: {len(df_results[df_results['price_change'] > 0])}")
print(f"  Decreases: {len(df_results[df_results['price_change'] < 0])}")
print(f"  No change: {len(df_results[df_results['price_change'] == 0])}")
print(f"\nAvg price change: {df_results['price_change_pct'].mean():.2f}%")


MODULE 2 SUMMARY

Total SKUs processed: 28794

By ABC Class:
abc_class
C    23438
B     4656
A      700

By Price Source:
price_source
capped_at_target+3std                   8854
margin                                  7603
unchanged (no tier below)               6306
unchanged                               5102
margin_max                               471
unchanged (at all_time_high ceiling)     364
all_time_high_margin                      69
unchanged (no tier above)                 25

Price Change Distribution:
  Increases: 4254
  Decreases: 3703
  No change: 20837

Avg price change: 0.22%


In [8]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
output_cols = [
    'product_id', 'warehouse_id', 'cohort_id', 'sku', 'brand', 'cat', 'abc_class', 'sensitivity',
    'stocks', 'zero_demand', 'combined_status', 'yesterday_status',
    'current_price', 'new_price', 'price_change', 'price_change_pct',
    'wac_p', 'current_margin', 'new_margin','current_cart_rule',
    'new_cart_rule', 'price_action', 'price_source', 'price_reason'
]

# Filter to only columns that exist
output_cols = [c for c in output_cols if c in df_results.columns]

# Drop duplicates before saving
df_output = df_results[output_cols].drop_duplicates(subset=['product_id', 'warehouse_id'], keep='first')
# Save df_output state before any manipulation for Slack upload later
temp_df_for_slack = df_output.copy()
print(f"\n✅ Saved {len(temp_df_for_slack)} rows for Slack upload")
print(f"Total records: {len(df_output)} (after removing {len(df_results) - len(df_output)} duplicates)")



✅ Saved 28794 rows for Slack upload
Total records: 28794 (after removing 0 duplicates)


In [9]:
# =============================================================================
# PUSH CART RULES & PRICES
# =============================================================================
# Push cart rules FIRST, then prices
# If cart rules fail for certain cohorts, skip those cohorts for prices

%run push_cart_rules_handler.ipynb
%run push_prices_handler.ipynb
pus = get_packing_units()

# ⚠️ MODE CONFIGURATION:
# - 'testing' (default): Prepare files but DON'T upload to API
# - 'live': Prepare files AND upload to MaxAB API
PUSH_MODE = 'live'  # Change to 'live' when ready to push

# =============================================================================
# STEP 1: Push Cart Rules First
# =============================================================================
print("\n" + "="*70)
print("STEP 1: PUSHING CART RULES")
print("="*70)

cart_result = push_cart_rules(df_output, pus, source_module='module_2', mode=PUSH_MODE)

print(f"\n{'='*60}")
print("CART RULES RESULT")
print(f"{'='*60}")
print(f"Mode: {cart_result['mode']}")
print(f"Cart rule changes: {cart_result['cart_rule_changes']}")
print(f"Pushed: {cart_result['pushed']}")
print(f"Failed: {cart_result['failed']}")
if cart_result['failed_cohorts']:
    print(f"⚠️ Failed cohorts: {cart_result['failed_cohorts']}")

# =============================================================================
# STEP 2: Push Prices (skip failed cohorts)
# =============================================================================
print("\n" + "="*70)
print("STEP 2: PUSHING PRICES")
print("="*70)

# Get failed cohorts from cart rules to skip in price push
failed_cohorts = cart_result.get('failed_cohorts', [])

# Call push_prices with the results, skipping failed cohorts
push_result = push_prices(df_output, pus, source_module='module_2', mode=PUSH_MODE, skip_cohorts=failed_cohorts)

print(f"\n{'='*60}")
print("PRICES RESULT")
print(f"{'='*60}")
print(f"Mode: {push_result['mode']}")
print(f"Source: {push_result['source_module']}")
print(f"Timestamp: {push_result['timestamp']}")
print(f"Total received: {push_result['total_received']}")
print(f"Price changes: {push_result['price_changes']}")
print(f"Pushed: {push_result['pushed']}")
print(f"Skipped: {push_result['skipped']}")
print(f"Failed: {push_result['failed']}")
if push_result.get('skipped_cohorts'):
    print(f"⚠️ Skipped cohorts (cart rules failed): {push_result['skipped_cohorts']}")


Push Cart Rules Handler loaded at 2026-02-18 08:10:27 Cairo time
✓ API credentials loaded successfully


Push Prices Handler loaded at 2026-02-18 08:10:27 Cairo time
✓ API credentials loaded successfully


✓ Google Sheets client initialized
Fetching packing_units ...


  Loaded 35273 records

STEP 1: PUSHING CART RULES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API

PUSH CART RULES - Source: module_2
Total received: 28794
Cart rule changes to push: 19163
Skipped (no change): 9631

Cart rule changes summary:
  Increases: 2609
  Decreases: 16554



📋 Prepared 20863 packing unit cart rules

Sample cart rule adjustments (showing products with multiple PUs):
 product_id  basic_unit_count  final_cart_rule  final_pu_cart_rule
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          3                 1               10                  10
          9                 1               10                  10

Processing cohort: 700


...

  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00,  3.70it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00,  3.69it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 9563
Total failed: 0

PRICES RESULT
Mode: live
Source: module_2
Timestamp: 2026-02-18 08:11:15
Total received: 28794
Price changes: 7957
Pushed: 9563
Skipped: 20837
Failed: 0


In [10]:
# =============================================================================
# UPLOAD RESULTS TO SNOWFLAKE AND SEND SLACK NOTIFICATION
# =============================================================================
from common_functions import upload_dataframe_to_snowflake, send_text_slack, send_file_slack

# Add created_at as DATE (module runs once per day at 8 AM)
df_output['created_at'] = datetime.now(CAIRO_TZ).date()

# Upload to Snowflake
print("\n" + "="*60)
print("UPLOADING RESULTS TO SNOWFLAKE")
print("="*60)

upload_status = upload_dataframe_to_snowflake(
    "Egypt", 
    df_output, 
    "MATERIALIZED_VIEWS", 
    "pricing_initial_push", 
    "append", 
    auto_create_table=True, 
    conn=None
)

# Prepare Slack notification
prices_pushed = push_result.get('pushed', 0)
prices_failed = push_result.get('failed', 0)
cart_rules_pushed = cart_result.get('pushed', 0)
cart_rules_failed = cart_result.get('failed', 0)

if upload_status:
    slack_message = f"""✅ *Module 2 - Initial Price Push Completed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Completed at: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
🔧 Mode: {PUSH_MODE.upper()}

📊 *Results:*
• Total SKUs processed: {len(df_output):,}
• Price changes: {push_result.get('price_changes', 0):,}
• Cart rule changes: {cart_result.get('cart_rule_changes', 0):,}

📤 *Push Status:*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed

🗄️ Results uploaded to: MATERIALIZED_VIEWS.pricing_initial_push"""
    
    send_text_slack('new-pricing-logic', slack_message)
    print("✅ Slack notification sent!")
    
    # Send output file to Slack after the text message (using saved copy before manipulation)
    SLACK_CHANNEL_ID = 'C0AAWK97Z3Q'
    send_file_slack(
        temp_df_for_slack, 
        f'📎 Module 2 Output: {len(temp_df_for_slack)} SKUs processed', 
        SLACK_CHANNEL_ID,
        filename=f'module2_initial_{datetime.now(CAIRO_TZ).strftime("%Y%m%d_%H%M")}.xlsx'
    )
    print("✅ Output file sent to Slack")
    
    print(f"✅ {len(df_output)} records uploaded to Snowflake")
else:
    error_message = f"""❌ *Module 2 - Initial Price Push Failed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Failed at: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
⚠️ Upload to Snowflake failed - please check logs

📤 *Push Status (before upload failure):*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed"""
    
    send_text_slack('new-pricing-logic', error_message)
    print("❌ Error notification sent to Slack!")
    
    # Still send output file even on error for debugging (using saved copy before manipulation)
    send_file_slack(
        temp_df_for_slack, 
        f'⚠️ Module 2 ERROR: {len(temp_df_for_slack)} SKUs', 
        SLACK_CHANNEL_ID,
        filename=f'module2_initial_ERROR_{datetime.now(CAIRO_TZ).strftime("%Y%m%d_%H%M")}.xlsx'
    )
    print("✅ Error file sent to Slack")



UPLOADING RESULTS TO SNOWFLAKE


/home/ec2-user/service_account_key.json


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://docs.slack.dev/tools/python-slack-sdk/v3-migration/
  warnings.warn(message)


/home/ec2-user/service_account_key.json


Message Sent
✅ Slack notification sent!


/home/ec2-user/service_account_key.json


File module2_initial_20260218_0813.xlsx sent to Slack
✅ Output file sent to Slack
✅ 28794 records uploaded to Snowflake
